In [5]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json

In [2]:
source = requests.get('https://www.btk.gov.tr/yillik-il-istatistikleri')

In [9]:
soup = BeautifulSoup(source.text, 'lxml')

In [24]:
files = soup.find_all(name='div', attrs={'class':'SmTqk'})[0]

In [29]:
links = {}
for a in files.find_all(name='a'):
    link = a['href']
    if link.endswith('.xlsx'):
        links[a.text] = link

In [30]:
links

{'2016 - 2021': 'https://www.btk.gov.tr/uploads/pages/yillik-il-istatistikleri/il-bazinda-yillik-istatistik-bulteni-v1.xlsx',
 '2015 - 2020': 'https://www.btk.gov.tr/uploads/pages/yillik-il-istatistikleri/elektronik-haberlesme-sektorune-iliskin-il-bazinda-yillik-istatistik-bulteni.xlsx',
 '2014 - 2019': 'https://www.btk.gov.tr/uploads/pages/yillik-il-istatistikleri/il-bazinda-yillik-istatistik-bulteni-20-05-2020.xlsx',
 '2013 - 2018': 'https://www.btk.gov.tr/uploads/pages/yillik-il-istatistikleri/il-bazinda-yillik-istatistik-bulteni1.xlsx',
 '2012 - 2017': 'https://www.btk.gov.tr/uploads/pages/yillik-il-istatistikleri/il-bazinda-yillik-istatistik-bulteni.xlsx',
 '2011 - 2016': 'https://www.btk.gov.tr/uploads/pages/slug/ehsyib-2011-2016-5a3a48147c468.xlsx',
 '2010 - 2015': 'https://www.btk.gov.tr/uploads/pages/slug/ehsyib-2010-2015.xlsx',
 '2009 - 2014': 'https://www.btk.gov.tr/uploads/pages/slug/ehsyib-2009-2014.xlsx',
 '2008 - 2013': 'https://www.btk.gov.tr/uploads/pages/slug/ehsyib-2

In [145]:
def get_dataframe(link):
    data = pd.read_excel(link, sheet_name='İl Bazlı Veri')
    data = data.iloc[2:,:]
    data['detail'] = data.iloc[:,1].fillna('') + data.iloc[:,2].fillna('')
    data['city'] = data.iloc[:,0]
    data = data.iloc[:,3:]
    data['city'] = data['city'].fillna(method='ffill')
    data.dropna(subset=[x for x in data.columns if str(x) not in ['city','detail']], axis=0, inplace=True)
    data['detail'] = data['detail'].str.strip(r'[()]')
    return data[data.columns[::-1]].reset_index(drop=True)[:-1]

In [162]:
df = pd.DataFrame()
for key, link in links.items():
    print(key)
    data = get_dataframe(link)
    if df.shape[0] == 0:
        df = data
    else:
        df = df.merge(data, how='left', on=['city', 'detail'])

2016 - 2021
2015 - 2020
2014 - 2019
2013 - 2018


C:\Users\TCLSOY~1\AppData\Local\Temp/ipykernel_5888/3319017312.py:8: FutureWarning: Passing 'suffixes' which cause duplicate columns {'2018_x', '2016_x', '2017_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = df.merge(data, how='left', left_on=['city', 'detail'], right_on=['city', 'detail'])


2012 - 2017


C:\Users\TCLSOY~1\AppData\Local\Temp/ipykernel_5888/3319017312.py:8: FutureWarning: Passing 'suffixes' which cause duplicate columns {'2015_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = df.merge(data, how='left', left_on=['city', 'detail'], right_on=['city', 'detail'])


2011 - 2016


...

C:\Users\TCLSOY~1\AppData\Local\Temp/ipykernel_5888/3319017312.py:8: FutureWarning: Passing 'suffixes' which cause duplicate columns {'2012_x', '2014_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = df.merge(data, how='left', left_on=['city', 'detail'], right_on=['city', 'detail'])


2008 - 2013


C:\Users\TCLSOY~1\AppData\Local\Temp/ipykernel_5888/3319017312.py:8: FutureWarning: Passing 'suffixes' which cause duplicate columns {'2011_x', '2013_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = df.merge(data, how='left', left_on=['city', 'detail'], right_on=['city', 'detail'])


2007 - 2012


C:\Users\TCLSOY~1\AppData\Local\Temp/ipykernel_5888/3319017312.py:8: FutureWarning: Passing 'suffixes' which cause duplicate columns {'2012_x', '2010_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = df.merge(data, how='left', left_on=['city', 'detail'], right_on=['city', 'detail'])


In [164]:
df.to_csv('final_merged.csv', encoding='utf-8-sig')

In [167]:
df.columns

Index([  'city', 'detail',     2021, '2020_x', '2019_x', '2018_x', '2017_x',
       '2016_x', '2020_y', '2019_y', '2018_y', '2017_y', '2016_y', '2015_x',
           2019, '2018_x', '2017_x', '2016_x', '2015_y', '2014_x', '2018_y',
       '2017_y', '2016_y', '2015_x', '2014_y', '2013_x',     2017, '2016_x',
       '2015_y', '2014_x', '2013_y', '2012_x', '2016_y', '2015_x', '2014_y',
       '2013_x', '2012_y', '2011_x', '2015_y', '2014_x', '2013_y', '2012_x',
       '2011_y', '2010_x', '2014_y', '2013_x', '2012_y', '2011_x', '2010_y',
       '2009_x', '2013_y', '2012_x', '2011_y', '2010_x', '2009_y', '2008_x',
       '2012_y',     2011, '2010_y',     2009, '2008_y',     2007],
      dtype='object')